In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\Usuario\Desktop\IA\CURSO\WEEK_4\DAY_2\EXERCICES\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [5]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [6]:
# Your code
from sklearn.model_selection import train_test_split

X = data ['text']
y = data ['label']

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2, random_state=42)

## Data Preprocessing

In [7]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
# Your code
import re

def clean_html(text):
    # remove scripts and styles
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.S)

    # remove comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.S)

    # remove all remaining tags
    text = re.sub(r'<.*?>', '', text)

    return text

data ['text'] = data['text'].apply(clean_html)
print(data)


                                                  text  label
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                             Will do.      0
2    Nora--Cheryl has emailed dozens of memos about...      0
3    Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                  fyi      0
..                                                 ...    ...
995  So what's the latest? It sounds contradictory ...      0
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...      1
997  Barb I will call to explain. Are you back in t...      0
998    Yang on travelNot free tonite.May work tomorrow      0
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...      0

[1000 rows x 2 columns]


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [9]:
# Your code
def clean_text_2(text):
    # Remove prefixed b
    text = re.sub(r"^b\s+", "", text)

    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Remove single characters
    text = re.sub(r'\b[a-zA-Z]\b', ' ', text)

    # Remove single characters from start
    text = re.sub(r'^\s*[a-zA-Z]\s+', ' ', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Convert to lowercase
    text = text.lower()

    return text.strip()

data ['text'] = data['text'].apply(clean_text_2)
print(data)

                                                  text  label
0    dear sir strictly private business proposal am...      1
1                                              will do      0
2    nora cheryl has emailed dozens of memos about ...      0
3    dear sir fmadam know that this proposal might ...      1
4                                                  fyi      0
..                                                 ...    ...
995  so what the latest it sounds contradictory and...      0
996  transfer of million pounds to youraccount my n...      1
997  barb will call to explain are you back in the ...      0
998    yang on travelnot free tonite may work tomorrow      0
999  sbwhoeopsunday february pmhshaunh just talked ...      0

[1000 rows x 2 columns]


## Now let's work on removing stopwords
Remove the stopwords.

In [10]:
# Your code
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

data ['text'] = data['text'].apply(remove_stopwords)
print(data)

                                                  text  label
0    dear sir strictly private business proposal mi...      1
1                                                           0
2    nora cheryl emailed dozens memos haiti weekend...      0
3    dear sir fmadam know proposal might surprise e...      1
4                                                  fyi      0
..                                                 ...    ...
995  latest sounds contradictory af decide shall ta...      0
996  transfer million pounds youraccount name mr ej...      1
997                     barb call explain back country      0
998       yang travelnot free tonite may work tomorrow      0
999  sbwhoeopsunday february pmhshaunh talked shaun...      0

[1000 rows x 2 columns]


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [11]:
# Your code
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [12]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

data ['text'] = data['text'].apply(lemmatize_text)
print(data)

                                                  text  label
0    dear sir strictly private business proposal mi...      1
1                                                           0
2    nora cheryl emailed dozen memo haiti weekend p...      0
3    dear sir fmadam know proposal might surprise e...      1
4                                                  fyi      0
..                                                 ...    ...
995  latest sound contradictory af decide shall tak...      0
996  transfer million pound youraccount name mr eji...      1
997                     barb call explain back country      0
998       yang travelnot free tonite may work tomorrow      0
999  sbwhoeopsunday february pmhshaunh talked shaun...      0

[1000 rows x 2 columns]


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [13]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

ham = data[data['label'] == 0]['text']
spam = data[data['label'] == 1]['text']

vectorizer = CountVectorizer()

def get_top_words(text_series, n=10):
    X = vectorizer.fit_transform(text_series)
    word_counts = X.toarray().sum(axis=0)
    words = vectorizer.get_feature_names_out()
    
    freq_df = pd.DataFrame({
        'word': words,
        'count': word_counts
    })
    
    return freq_df.sort_values(by='count', ascending=False).head(n)

top_ham = get_top_words(ham, 10)
top_spam = get_top_words(spam, 10)

print("Top 10 words in HAM:")
print(top_ham)

print("\nTop 10 words in SPAM:")
print(top_spam)

Top 10 words in HAM:
           word  count
5416      state    136
4302         pm    127
6463      would    107
4433  president     99
5855       time     95
883        call     94
3688         mr     91
3902      obama     84
4219    percent     81
5058  secretary     79

Top 10 words in SPAM:
              word  count
13872        money    981
230        account    895
2403          bank    800
7931          fund    781
21155  transaction    549
3170      business    514
4481       country    508
13993           mr    489
14369         nbsp    475
13682      million    460


## Extra features

In [14]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

data_train['preprocessed_text'] = data_train['text'].apply(lemmatize_text).astype(str)
data_val['preprocessed_text']   = data_val['text'].apply(lemmatize_text).astype(str)



data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
29,regard mr nelson smith kindly reply private em...,1,regard mr nelson smith kindly reply private em...,0,0,79
535,able reach oscar supposed send pdb receive,0,able reach oscar supposed send pdb receive,0,0,42
695,huma abedin checking pat work jack jake rest a...,0,huma abedin checking pat work jack jake rest a...,0,0,76
557,announced monday today,0,announced monday today,0,0,22
836,bank africaagence san pedro bp san pedro cote ...,1,bank africaagence san pedro bp san pedro cote ...,1,1,1050


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

X_train_bow = vectorizer.fit_transform(data_train['preprocessed_text'])

X_val_bow = vectorizer.transform(data_val['preprocessed_text'])

print(X_train_bow.shape)



(800, 27977)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [18]:
# Your code
vectorizer = TfidfVectorizer(stop_words='english')

X_tfidf = vectorizer.fit_transform(data['text'])

print(X_tfidf.shape)

(1000, 29742)


## And the Train a Classifier?

In [ ]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english", max_features=5000)

X_train_tfidf = tfidf.fit_transform(data_train["preprocessed_text"])
X_val_tfidf   = tfidf.transform(data_val["preprocessed_text"])

import numpy as np
from scipy.sparse import hstack

extra_cols = ["money_mark", "suspicious_words", "text_len"]

X_train_extra = data_train[extra_cols].values
X_val_extra   = data_val[extra_cols].values

# Combine sparse TF-IDF and dense extra features
X_train_all = hstack([X_train_tfidf, X_train_extra])
X_val_all   = hstack([X_val_tfidf, X_val_extra])

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_all, data_train["label"])

y_pred = clf.predict(X_val_all)

print("Accuracy:", accuracy_score(data_val["label"], y_pred))
print(confusion_matrix(data_val["label"], y_pred))
print(classification_report(data_val["label"], y_pred))


Accuracy: 0.95
[[122   3]
 [  7  68]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       125
           1       0.96      0.91      0.93        75

    accuracy                           0.95       200
   macro avg       0.95      0.94      0.95       200
weighted avg       0.95      0.95      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code